In [1]:
# LIBRARIES IMPORT ------------------------------------------------
import pandas as pd
import numpy as np
import datetime
import re

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.layers import Dense, LSTM, TimeDistributed, Input

from os.path import join

import warnings
warnings.filterwarnings("ignore")

# function for RMSLE
def rmsle(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)**0.5

In [2]:
# DATA DOWNLOAD ========================================================   
# air reservation system
data_path = 'c:/data/'
air_reserve = pd.read_csv(data_path + 'air_reserve.csv')
hpg_reserve = pd.read_csv(data_path + 'hpg_reserve.csv')
air_store_info = pd.read_csv(data_path + 'air_store_info.csv')
hpg_store_info = pd.read_csv(data_path + 'hpg_store_info.csv')
air_visit_data = pd.read_csv(data_path + 'air_visit_data.csv')
store_id_relation = pd.read_csv(data_path + 'store_id_relation.csv')
date_info = pd.read_csv(data_path + 'date_info.csv')
sample_sub = pd.read_csv(data_path + 'sample_submission.csv')

In [3]:
date_info

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0
...,...,...,...
512,2017-05-27,Saturday,0
513,2017-05-28,Sunday,0
514,2017-05-29,Monday,0
515,2017-05-30,Tuesday,0


In [5]:
col_name = ['golden_week_days']
list1 = ['2016-04-29','2016-05-03','2016-05-04','2016-05-05','2017-04-29','2017-05-03','2017-05-04','2017-05-05']
gold = pd.DataFrame({'calendar_date' : ['2016-04-29','2016-05-03','2016-05-04','2016-05-05','2017-04-29','2017-05-03','2017-05-04','2017-05-05'],
                     'golden_week_days' : [1, 1, 1, 1, 1, 1, 1,1]})
gold = gold.astype({'golden_week_days': 'int'})

In [6]:
merge = pd.merge(date_info, gold, how='outer', on='calendar_date')
cleaned = merge.dropna()
cleaned

,calendar_date,day_of_week,holiday_flg,golden_week_days
119,2016-04-29,Friday,1,1.0
123,2016-05-03,Tuesday,1,1.0
124,2016-05-04,Wednesday,1,1.0
125,2016-05-05,Thursday,1,1.0
484,2017-04-29,Saturday,1,1.0
488,2017-05-03,Wednesday,1,1.0
489,2017-05-04,Thursday,1,1.0
490,2017-05-05,Friday,1,1.0


In [7]:
golden_week_days = merge.fillna(0)
golden_week_days = golden_week_days.astype({'golden_week_days': 'int64'})
golden_week_days

,calendar_date,day_of_week,holiday_flg,golden_week_days
0,2016-01-01,Friday,1,0
1,2016-01-02,Saturday,1,0
2,2016-01-03,Sunday,1,0
3,2016-01-04,Monday,0,0
4,2016-01-05,Tuesday,0,0
...,...,...,...,...
512,2017-05-27,Saturday,0,0
513,2017-05-28,Sunday,0,0
514,2017-05-29,Monday,0,0
515,2017-05-30,Tuesday,0,0


In [8]:
golden_week_days.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 0 to 516
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   calendar_date     517 non-null    object
 1   day_of_week       517 non-null    object
 2   holiday_flg       517 non-null    int64 
 3   golden_week_days  517 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 20.2+ KB
